In [1]:
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GroupKFold

In [ ]:
df_train =  pd.read_feather('train_24cols.feather')


In [3]:
df_train = df_train.fillna(0)
df_train['trade_seconds'] = 'more'



for val in range(3): df_train.loc[df_train.seconds_in_bucket_size_0_600==val, 'trade_seconds'] = val

In [4]:
kfold = GroupKFold()


In [5]:
df_train.stock_id = df_train.stock_id.astype('category')

In [6]:
cont_nn,cat_nn = cont_cat_split(df_train, max_card=9000, dep_var='target')

In [7]:
cat_nn.remove('time_id'), cat_nn.remove('row_id')

(None, None)

In [11]:
def rmspe(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [18]:
def train_fold(trn_idx, val_idx, fname):
    

    procs_nn = [Categorify, Normalize]
    to_nn = TabularPandas(df_train, procs_nn, cat_nn, cont_nn,
                          splits=[list(trn_idx), list(val_idx)], y_names='target')
    config={'lin_first':False, 'embed_p':.15, 'ps':[0,.4,.1], 'act_cls':nn.SiLU(inplace=True)}
    dls = to_nn.dataloaders(1024)
    learn = tabular_learner(dls, y_range=(0,.1), layers=[600,300,150], config = config,
                        n_out=1, loss_func=rmspe, metrics=AccumMetric(rmspe),
                        
                           )
    print(learn.model)
    learn.fit_flat_cos(30, 5e-3)
    learn.save(fname)
    return learn.recorder.metrics[0].value

In [19]:
res =[]
for trn_idx, val_idx in kfold.split(df_train, groups = df_train.time_id):
    res.append(train_fold(trn_idx, val_idx, f'tuned_{len(res)}'))
    print('fold', len(res), res[-1])
np.mean(res),res

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(113, 23)
    (1): Embedding(5, 4)
  )
  (emb_drop): Dropout(p=0.15, inplace=False)
  (bn_cont): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(47, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=47, out_features=600, bias=False)
      (2): SiLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.4, inplace=False)
      (2): Linear(in_features=600, out_features=300, bias=False)
      (3): SiLU(inplace=True)
    )
    (2): LinBnDrop(
      (0): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): Linear(in_features=300, out_features=150, bias=False)
      (3): SiLU(inplace=True)
    )
    (3): LinBnDrop(
    

epoch,train_loss,valid_loss,rmspe,time
0,0.319939,0.255470,0.258348,00:02


KeyboardInterrupt: 